In [ ]:
import numpy as np
import sympy as sp
from sympy import symbols, log, lambdify, primitive
from matplotlib import pyplot as plt
from symbolic_hulls_func_aggr import *

In [ ]:
from sympy import primitive

def taylor_approximation(f, x, a):
    """
    Computes the Taylor series approximation of f up to degree 2 centered at x = a.
    
    Parameters:
    f : sympy expression
        The function to approximate.
    x : sympy symbol
        The variable in the function.
    a : value
        The point around which to expand the function.
    
    Returns:
    sympy expression
        The quadratic Taylor approximation.
    """
    f_a = f.subs(x, a)
    f_prime = sp.diff(f, x).subs(x, a) * (x - a)
    f_double_prime = sp.diff(f, x, x).subs(x, a) * (x - a)**2 / 2
    
    return f_a + f_prime + f_double_prime

def transform_diff(f1, f2):       
    free_symbols_f1 = f1.free_symbols
    free_symbols_f2 = f2.free_symbols

    variables = tuple(free_symbols_f1.intersection(free_symbols_f2))
    pvariables = tuple(sp.symbols(f'{symbol}_p') for symbol in variables)

    pvariables_dict = dict(zip(variables, pvariables))

    pgrad_f1 = gradient(f1.subs(pvariables_dict), pvariables)
    
    transform_1 = td_legendre_transform(variables, pgrad_f1, f2)
    transform_2 = td_legendre_transform(pvariables, pgrad_f1, f1.subs(pvariables_dict))

    return primitive(transform_1 - transform_2)[1]

In [ ]:
x = symbols('x')
f1 = 2*x*log(x)
f2 = (2-x)*log(2-x)+ .5

f1_numeric = lambdify(x, f1, 'numpy')
f2_numeric = lambdify(x, f2, 'numpy')

In [ ]:
# Boudnary 1
proj = transform_diff(f1, f2)
a = sp.solve(sp.Eq(sp.diff(proj, x), 0))[0][x]
proj_approx = taylor_approximation(proj, x, a)
boundary1 = sp.discriminant(proj_approx, x)
boundary1_numeric = lambdify(boundary1.free_symbols.pop(), boundary1, 'numpy')

In [ ]:
# Boudnary 2
proj = transform_diff(f2, f1)
a = sp.solve(sp.Eq(sp.diff(proj, x), 0))[0][x]
proj_approx = taylor_approximation(proj, x, a)
boundary2 = sp.discriminant(proj_approx, x)
boundary2_numeric = lambdify(boundary2.free_symbols.pop(), boundary2, 'numpy')

In [ ]:
# Solve the boundaries
from scipy.optimize import fsolve

x1 = fsolve(boundary1_numeric, 0.5)
x2 = fsolve(boundary2_numeric, 1.7)

print(x1, x2)

In [ ]:
# Define x range while avoiding log domain issues
x_vals = np.linspace(0.01, 1.99, 400)  # Avoid x=0 and x=2 where log is undefined

# Compute y values
y1_vals = f1_numeric(x_vals)
y2_vals = f2_numeric(x_vals)
y3_vals = boundary1_numeric(x_vals)
y4_vals = boundary2_numeric(x_vals)

# Plot the functions
plt.figure(figsize=(8, 6))
plt.plot(x_vals, y1_vals, label=r'$f_1(x) = 2x \ln(x)$', color='blue')
plt.plot(x_vals, y2_vals, label=r'$f_2(x) = (2-x) \ln(2-x) + 0.5$', color='red')
# plt.plot(x_vals, y3_vals, label=r'projection 1', color='green')
# plt.plot(x_vals, y4_vals, label=r'projection 2', color='green')

# Plot the boundaries
plt.scatter(x1, f1_numeric(x1), color='black', zorder=5)
plt.scatter(x2, f2_numeric(x2), color='black', zorder=5)

# Formatting the plot
plt.xlabel('x')
plt.ylabel('f(x)')
plt.title('Plot of $f_1(x)$ and $f_2(x)$')
plt.axhline(0, color='black', linewidth=0.5, linestyle='--')
plt.axvline(0, color='black', linewidth=0.5, linestyle='--')
plt.xlim(0, 2)
plt.ylim(-1, 1)
plt.legend()
plt.grid(True)

# Show the plot
plt.show()


## Here I am going to try to see if the math simplifies

In [ ]:
import sympy as sp

x, t = sp.symbols('x, t')
f1 = sp.Function('f_1')(x)
f2 = sp.Function('f_2')(x)

f_sol = sp.Function('f_sol')(t)

In [ ]:
proj = f2 - f1.subs(x, t) - sp.Derivative(f1.subs(x, t), t) * (x - t)

In [ ]:
proj_taylor = proj.subs(x, f_sol) + sp.Derivative(proj, x).subs(x, f_sol) * (x - f_sol) + sp.Derivative(proj, x, x).subs(x, f_sol) * (x - f_sol)**2 / 2
proj_taylor

In [ ]:
A = sp.expand(proj_taylor).coeff(x, 2)
B = sp.expand(proj_taylor).coeff(x, 1)
C = sp.expand(proj_taylor).coeff(x, 0)

In [ ]:
disc = B**2 - 4*A*C
disc

In [ ]:
disc_simple = sp.simplify(sp.Eq(disc, 0))
disc_simple